In [44]:
rm(list = ls())

In [45]:
# libraries
suppressMessages({
  library(tidyverse)
  library(broom)
  library(ranger)
}
)

In [46]:
Data <- read.csv("/Users/mpaga/Downloads/train.csv",sep=",")

In [47]:
dim(Data)

[1] 1460   81

In [48]:
names(Data)

 [1] "Id"            "MSSubClass"    "MSZoning"      "LotFrontage"   "LotArea"      
 [6] "Street"        "Alley"         "LotShape"      "LandContour"   "Utilities"    
[11] "LotConfig"     "LandSlope"     "Neighborhood"  "Condition1"    "Condition2"   
[16] "BldgType"      "HouseStyle"    "OverallQual"   "OverallCond"   "YearBuilt"    
[21] "YearRemodAdd"  "RoofStyle"     "RoofMatl"      "Exterior1st"   "Exterior2nd"  
[26] "MasVnrType"    "MasVnrArea"    "ExterQual"     "ExterCond"     "Foundation"   
[31] "BsmtQual"      "BsmtCond"      "BsmtExposure"  "BsmtFinType1"  "BsmtFinSF1"   
[36] "BsmtFinType2"  "BsmtFinSF2"    "BsmtUnfSF"     "TotalBsmtSF"   "Heating"      
[41] "HeatingQC"     "CentralAir"    "Electrical"    "X1stFlrSF"     "X2ndFlrSF"    
[46] "LowQualFinSF"  "GrLivArea"     "BsmtFullBath"  "BsmtHalfBath"  "FullBath"     
[51] "HalfBath"      "BedroomAbvGr"  "KitchenAbvGr"  "KitchenQual"   "TotRmsAbvGrd" 
[56] "Functional"    "Fireplaces"    "FireplaceQu"   "GarageType"

In [49]:
#remove Id colum
Data["Id"] <- NULL
dim(Data)

[1] 1460   80

In [50]:
# numeric data
numData <- Data |> 
  select_if(is.numeric) 

In [51]:
hist(Data$SalePrice)

In [52]:
summary(Data$SalePrice)

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
  34900  129975  163000  180921  214000  755000 

In [53]:
skimr::skim_without_charts(Data,where(is.numeric))-> num_skimData

In [54]:
(numVal_na <- num_skimData$skim_variable[num_skimData$n_missing !=0] )

[1] "LotFrontage" "MasVnrArea"  "GarageYrBlt"

In [55]:
skimr::skim_without_charts(Data,where(is.character))-> cat_skimData

In [56]:
summary(cat_skimData)

── Data Summary ────────────────────────
                           Values
Name                       Data  
Number of rows             1460  
Number of columns          80    
_______________________          
Column type frequency:           
  character                43    
________________________         
Group variables            None  

Let's impute some of these numerical features

In [57]:
colMeans(is.na(Data[numVal_na]))

LotFrontage  MasVnrArea GarageYrBlt 
0.177397260 0.005479452 0.055479452 

In [58]:
# list of num featute to impute
imputeVal_list <- apply(Data[numVal_na],2, FUN = "median",na.rm = T,simplify = list)

In [59]:
#impute numerical features 
Data[numVal_na] <- replace_na(Data[numVal_na] ,
                              replace = imputeVal_list
)

In [60]:
# check  na
colMeans(is.na(Data[numVal_na]))

LotFrontage  MasVnrArea GarageYrBlt 
          0           0           0 

In [61]:
Data |> 
  select_if(is.numeric) |> 
  unique() |> 
  nrow()

[1] 1460

In [62]:
Data |> 
  select_if(is.numeric)  ->numData 
  lm(SalePrice~.,numData) |> 
    summary() |> 
    tidy() |> 
    filter(p.value <=0.5) |> 
    nrow()

[1] 28

28 numerical features have predictive effect on target feature. 

In [63]:
# correlation 
for (feature in names(numData)[-1]){
  if (cor(numData[names(numData)][1],numData[feature]) >= 0.8 ) print(feature)
}

There is no carrelated features in numeric features 

cat var feat engineering  

In [64]:
# n cat features presenting missing values 
sum(cat_skimData$n_missing  !=0)

[1] 16

In [96]:
table(Data[,feature])


Gar2 Othr Shed TenC 
   2    2   49    1 

In [95]:
# median impute 
cat_features_na <- cat_skimData$skim_variable[cat_skimData$n_missing !=0]

for (feature in cat_features_na){
  replace_na(Data[,feature],
  replace = names(table(Data[,feature]))[1]
)
}
colMeans(is.na(Data[cat_features_na]))


       Alley   MasVnrType     BsmtQual     BsmtCond BsmtExposure BsmtFinType1 BsmtFinType2 
0.9376712329 0.0054794521 0.0253424658 0.0253424658 0.0260273973 0.0253424658 0.0260273973 
  Electrical  FireplaceQu   GarageType GarageFinish   GarageQual   GarageCond       PoolQC 
0.0006849315 0.4726027397 0.0554794521 0.0554794521 0.0554794521 0.0554794521 0.9952054795 
       Fence  MiscFeature 
0.8075342466 0.9630136986 

In [71]:
colMeans(is.na(Data[cat_features_na]))

       Alley   MasVnrType     BsmtQual     BsmtCond BsmtExposure BsmtFinType1 BsmtFinType2 
0.9376712329 0.0054794521 0.0253424658 0.0253424658 0.0260273973 0.0253424658 0.0260273973 
  Electrical  FireplaceQu   GarageType GarageFinish   GarageQual   GarageCond       PoolQC 
0.0006849315 0.4726027397 0.0554794521 0.0554794521 0.0554794521 0.0554794521 0.9952054795 
       Fence  MiscFeature 
0.8075342466 0.9630136986 

16 features have missing values or NA

let's use random forest to predict missing values 

to be continued 😉!